# Reproduce Figure 2
- Py kernel

In [1]:
import pandas as pd
import numpy as np
from scipy import stats
import matplotlib.pyplot as plt
import seaborn as sns

import sys
sys.path.insert(1, '/Users/brinkley97/Documents/development/')
import my_created_functions

# Load Data

In [2]:
path_to_data =  "lab-kcad/datasets/tiles_dataset/" 
work_data = "figure_2/physical/diurnal_work_lm_6.csv.gz"
off_data = "figure_2/physical/diurnal_off_lm_6.csv.gz"

In [3]:
work_df = my_created_functions.load_gzip_csv_data(path_to_data, work_data)
off_df = my_created_functions.load_gzip_csv_data(path_to_data, off_data)

# Modify Pandas DataFrames

In [4]:
def modify_df(work_df, off_df):
    '''Modify work and off terminology (Dataframe column headings and values) to match those in the specific questions
    
    Arguments:
    work_df -- Pandas Dataframe
    off_df -- Pandas Dataframe
    
    Return: 
    updated work df, updated off df, a combination of both -- Pandas Dataframes in tuple
    '''

    day_map = {'4th': "11:00–15:00 h", '5th': "15:00–19:00 h", '6th': "19:00–23:00 h", '1st': "23:00–3:00 h", '2nd': "3:00–7:00 h", '3rd': "7:00–11:00 h"}
    night_map = {'1st': '23:00–3:00 h', '2nd': '3:00–7:00 h', '3rd': '7:00–11:00 h', '4th': '11:00–15:00 h', '5th': '15:00–19:00 h', '6th': '19:00–23:00 h'}
    
    work_df_copy = work_df.copy()
    work_df_copy = work_df_copy.drop(columns=["id", "age",  "gender"])
    update_work_df = work_df_copy.replace({"time": day_map})
    
    off_df_copy = off_df.copy()
    off_df_copy = off_df_copy.drop(columns=["id", "age",  "gender"])
    update_off_df = off_df_copy.replace({"time": night_map})

    combine_dfs = pd.concat([update_work_df, update_off_df])
    combine_dfs = combine_dfs.reset_index(drop=True)
    return update_work_df, update_off_df, combine_dfs

In [5]:
modified_work_df, modified_off_df, combine_dfs = modify_df(work_df, off_df)

# Load Generated Specific Questions

In [6]:
base = "/Users/brinkley97/Documents/development/lab-kcad/"
generated_sq_file = "TGN10Plus/generateSpecificQuestions.ipynb"
figure_2_specific_questions = base + generated_sq_file

In [7]:
# %load figure_2_specific_questions
/Users/brinkley97/Documents/development/lab-kcad/TGN10Plus/generateSpecificQuestions.ipynb

In [8]:
%run "../generateSpecificQuestions.ipynb"

In [10]:
# figure_2_sqs

# Run Analysis

## Create Plots

In [10]:
def create2_plot(night_shift_at_work, day_shift_at_work, night_shift_off_work, day_shift_off_work, ontology_to_find, time):
    '''
    '''
    plt_df = pd.concat([night_shift_at_work, day_shift_at_work, night_shift_off_work, day_shift_off_work])
    plt_df = plt_df.reset_index()
    # print(plt_df)
    data = []
    if ontology_to_find == "rest":
        p = sns.barplot(x="work", y="rest", data=plt_df, hue="shift", palette="seismic")
        # print(ontology_to_find)
        for value in p.containers:
            p.bar_label(value, )
            # print(p.bar_label(value, ))
        p.set(title=time)
        plt.show()
        print()
        
    elif ontology_to_find == "step_ratio":
        p = sns.barplot(x="work", y="step_ratio", data=plt_df, hue="shift", palette="seismic")
        # print(ontology_to_find)
        for value in p.containers:
            p.bar_label(value, )
            # print(p.bar_label(value), )
        p.set(title=time)
        plt.show()
        print()
        
    
    return 

## Independent TTest - Figure 2

In [66]:
def calculate_ttest_ind(f2_specific_question, split_f2_sq_into_words, work_df, off_df, ontology_to_find):
    '''Calculate the p-value for... 
        - hours-work-day-rest
        - hours-off-day-rest
        
        - hours-work-night-rest
        - hours-off-night-rest
        
        - hours-work-day-step_ratio
        - hours-off-day-step_ratio
        
        - hours-work-night-step_ratio
        - hours-off-night-step_ratio
        
    Run "code/physical/plot_diurnal_data.py" to check results
    
    Arguments:
    f2_specific_question -- string
    split_f2_sq_into_words -- list
    work_df -- Pandas Dataframe
    off_df -- Pandas Dataframe
    ontology_to_find -- string
    
    Return:
    p-values -- Pandas Dataframe
    '''
    
    print(f2_specific_question)
    filter_shifts = ["day", "night"]
    filter_times = list(ontology_mappings.values())[5]
    
    # empty; will fill later
    store_p_values_in_df = pd.DataFrame(index=filter_times)
    
    group_by_time_work_df = work_df.set_index(["time"])
    group_by_time_off_df = off_df.set_index(["time"])
   
    for time_idx in range(len(split_f2_sq_into_words)):
        
        time = split_f2_sq_into_words[time_idx]
        if time in filter_times:
            
            '''
            Work DF
            '''
            work_by_time = group_by_time_work_df.loc[time]
            # print("\n", work_by_time, "---")
            
            day_shift_at_work = work_by_time.loc[work_by_time["shift"] == "day"]
            # print(time, "\n", "work", "day", day_shift_at_work)
                
            night_shift_at_work = work_by_time.loc[work_by_time["shift"] == "night"]
            # print(time, "\n", "work", "night", night_shift_at_work)
                
            # meadian_day, mean_day = np.round(np.nanmedian(work_shift_day[ontology_to_find]) * 100, 6), np.round(np.nanmean(work_shift_day[ontology_to_find]) * 100, 6)
            # print("work", "day", time, meadian_day, mean_day)
            
            # meadian_night, mean_night = np.round(np.nanmedian(work_shift_day[ontology_to_find]) * 100, 6), np.round(np.nanmean(work_shift_day[ontology_to_find]) * 100, 6)
            # print("work", "night", time, meadian_day, mean_night)
            
            stats_value, work_p_value = stats.ttest_ind(np.array(day_shift_at_work[ontology_to_find]), np.array(night_shift_at_work[ontology_to_find]))
            # print("work", time, work_p_value)
            
            if work_p_value < 0.01:
                print("work", time, work_p_value, '**')
            elif work_p_value < 0.05:
                print("work", time, work_p_value, '*')
            else:
                print("work", time, work_p_value)
            
            '''
            Off DF
            '''
            off_by_time = group_by_time_off_df.loc[time]
            # print("\n", off_by_time, "---")
                
            day_shift_off_work = off_by_time.loc[off_by_time["shift"] == "day"]
            # print(time, "\n", "off", "day", day_shift_off_work)
                
            night_shift_off_work = off_by_time.loc[off_by_time["shift"] == "night"]
            # print(time, "\n", "off", "night", night_shift_off_work)
            
            stats_value, off_p_value = stats.ttest_ind(np.array(day_shift_off_work[ontology_to_find]), np.array(night_shift_off_work[ontology_to_find]))
            
            if off_p_value < 0.01:
                print("off", time, off_p_value, '**')
            elif off_p_value < 0.05:
                print("off", time, off_p_value, '*')
            else:
                print("off", time, off_p_value) 
               
            '''
            Create Plot
            '''
            # create2_plot(night_shift_at_work, day_shift_at_work, night_shift_off_work, day_shift_off_work, ontology_to_find, time)
            
            # df = create_plot(work_by_time, off_by_time, ontology_to_find)
            # print(df)
            
            store_p_values_in_df.loc[time, "p-val"] = work_p_value
            store_p_values_in_df.loc[time, "p-val"] = off_p_value
            print()
            
    print("\n------------------------------------------\n")
    
    return store_p_values_in_df

In [67]:
def figure_two(figure_2_specific_questions, work_df, off_df):
    '''Reproduce Table 2
    
    Arguments:
    figure_2_specific_questions -- list
    work_df -- Dataframe
    off_df -- Dataframe
    
    Function:
    calculate_ttest_ind()
    
    Return:
    nothing; everything is being printed in calculate_ttest_ind()
    '''
   
    filter_physiological = ["rest", "step_ratio"]
    
    for figure_2_specific_question_idx in range(len(figure_2_specific_questions)):
        f2_specific_question = figure_2_specific_questions[figure_2_specific_question_idx]
        split_f2_sq_into_words = f2_specific_question.split("*")
        
        work_df = work_df.reset_index(drop=True)        
        off_df = off_df.reset_index(drop=True)
        
        for filter_physiological_idx in range(len(filter_physiological)):
            
            ontology_to_find = filter_physiological[filter_physiological_idx]
            
            # only get f2's specific questions that include rest and step_ratio (per the original analysis study)
            if ontology_to_find in split_f2_sq_into_words:
                
                work_df["type"] = ontology_to_find
                work_df[ontology_to_find] = work_df[ontology_to_find]
                modified_work_df = work_df.loc[0:, ["work", "time", ontology_to_find, "shift", "type"]]
                
                off_df["type"] = ontology_to_find
                off_df[ontology_to_find] = off_df[ontology_to_find]
                modified_off_df = off_df.loc[0:, ["work", "time", ontology_to_find, "shift", "type"]]
                
                p_value = calculate_ttest_ind(f2_specific_question, split_f2_sq_into_words, modified_work_df, modified_off_df, ontology_to_find)
                # print(p_val)
    return 

In [68]:
figure_two(figure_2_sqs, modified_work_df, modified_off_df)

on both *work_1* day and *work_2* day, what is the effect of primarily *day-shift* nurses and primarily *night-shift* nurses on *rest* between the hours of *23:00–3:00 h* ?
work 23:00–3:00 h 0.0015494236117052733 **
off 23:00–3:00 h 0.8501148450822772


------------------------------------------

on both *work_1* day and *work_2* day, what is the effect of primarily *day-shift* nurses and primarily *night-shift* nurses on *rest* between the hours of *3:00–7:00 h* ?
work 3:00–7:00 h 0.034467656157217745 *
off 3:00–7:00 h 0.3176447299854959


------------------------------------------

on both *work_1* day and *work_2* day, what is the effect of primarily *day-shift* nurses and primarily *night-shift* nurses on *rest* between the hours of *7:00–11:00 h* ?
work 7:00–11:00 h 8.619878000514936e-08 **
off 7:00–11:00 h 4.227081558573983e-05 **


------------------------------------------

on both *work_1* day and *work_2* day, what is the effect of primarily *day-shift* nurses and primarily *